<a href="https://colab.research.google.com/github/shaya-686/AI/blob/main/%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D1%87%D0%BD%D0%B0_recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Завдання

* Створіть Reader
* Створіть датасет та розділіть його на тренувальні та тестові дані
* Виберіть метрики для поріняння якості моделей
* На основі метрик виберіть найкращу модель
* Використайте `optuna` для підбору найкращих параметрів



In [1]:
import pandas as pd

df = pd.read_csv("https://raw.githubusercontent.com/HalyshAnton/IT-Step-Pyton-AI/main/module7/data/Blog%20Ratings.csv")

df.head()

,blog_id,userId,ratings
0,9025,11,3.5
1,9320,11,5.0
2,9246,11,3.5
3,9431,11,5.0
4,875,11,2.0


In [2]:
df.describe()

,blog_id,userId,ratings
count,200140.000000,200140.000000,200140.000000
mean,5652.533621,2545.710158,3.117468
std,2970.685946,1446.195478,1.768113
min,1.000000,10.000000,0.500000
25%,2906.000000,1314.000000,2.000000
50%,5994.000000,2552.000000,3.500000
75%,8510.000000,3795.000000,5.000000
max,9755.000000,5010.000000,5.000000


In [6]:
df.shape

(200140, 3)

In [3]:
!pip install -q surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 1.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [7]:
from surprise import Dataset, SVD, Reader

reader = Reader(rating_scale=(0.5, 5))

# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(df[["userId", "blog_id", "ratings"]], reader)

In [8]:
from surprise.model_selection import train_test_split

trainset, testset = train_test_split(data, train_size=0.8)

In [19]:
from surprise import BaselineOnly

bsl_options = {'method': 'sgd',
               'reg_u': 0.0001,
               'reg_i': 0.0001}

algo = BaselineOnly(bsl_options=bsl_options)
algo.fit(trainset)

Estimating biases using sgd...


In [10]:
from surprise import accuracy

In [20]:
preds = algo.test(testset)

print(f"mae = {accuracy.mae(preds, verbose=False)}")
print(f"mse = {accuracy.mse(preds, verbose=False)}")
print(f"rmse= {accuracy.rmse(preds, verbose=False)}")
print(f"fcp = {accuracy.fcp(preds, verbose=False)}")

mae = 1.6239103709037646
mse = 3.2944523607321012
rmse= 1.815062632729819
fcp = 0.5048351762127237


In [27]:
from surprise import SVD

algo = SVD(n_factors=150,
           n_epochs=50,
           )

algo.fit(trainset)

In [28]:
preds = algo.test(testset)

print(f"mae = {accuracy.mae(preds, verbose=False)}")
print(f"mse = {accuracy.mse(preds, verbose=False)}")
print(f"rmse= {accuracy.rmse(preds, verbose=False)}")
print(f"fcp = {accuracy.fcp(preds, verbose=False)}")

mae = 1.6566768720352663
mse = 3.573701134259359
rmse= 1.8904235330368058
fcp = 0.4941640422963094


In [29]:
from surprise import KNNBasic

algo = KNNBasic(k=15,
                min_k=5,
                sim_options={'name': 'cosine',
                             'user_based': True})

algo.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [30]:
preds = algo.test(testset)

print(f"mae = {accuracy.mae(preds, verbose=False)}")
print(f"mse = {accuracy.mse(preds, verbose=False)}")
print(f"rmse= {accuracy.rmse(preds, verbose=False)}")
print(f"fcp = {accuracy.fcp(preds, verbose=False)}")

mae = 1.6322712417052745
mse = 3.377646429342832
rmse= 1.8378374327842035
fcp = 0.5086306078252204


In [17]:
from surprise import CoClustering

algo = CoClustering(n_cltr_u = 5,
                    n_cltr_i = 15,
                    n_epochs = 40)

algo.fit(trainset)

In [18]:
preds = algo.test(testset)

print(f"mae = {accuracy.mae(preds, verbose=False)}")
print(f"mse = {accuracy.mse(preds, verbose=False)}")
print(f"rmse= {accuracy.rmse(preds, verbose=False)}")
print(f"fcp = {accuracy.fcp(preds, verbose=False)}")

mae = 1.7364487035648672
mse = 4.163777770184704
rmse= 2.040533697390147
fcp = 0.49867618683366377


In [39]:
from sklearn import metrics


def objective(trial):


  params = {
        'n_factors': trial.suggest_int('n_factors', 10, 100),
        'n_epochs': trial.suggest_int('n_epochs', 10, 50),
        'lr_all': trial.suggest_float('lr_all', 0.001, 0.1),
        'reg_all': trial.suggest_float('reg_all', 0.1, 1.0)
    }


  model = SVD(**params)

  model.fit(trainset)

  preds = model.test(testset)
  mae = accuracy.mae(preds, verbose=False)

  return mae

In [32]:
!pip install -q optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 4.3 MB/s eta 0:00:00


In [40]:
import optuna
study = optuna.create_study(directions=['minimize'])
study.optimize(objective, n_trials=20)

[I 2024-08-29 17:52:00,247] A new study created in memory with name: no-name-4c5eadba-fcc6-43a2-9b42-ba275af3b3df
[I 2024-08-29 17:52:03,829] Trial 0 finished with value: 1.6157304932293513 and parameters: {'n_factors': 74, 'n_epochs': 18, 'lr_all': 0.01854043067045209, 'reg_all': 0.9931401758273574}. Best is trial 0 with value: 1.6157304932293513.
[I 2024-08-29 17:52:08,737] Trial 1 finished with value: 1.63533256936309 and parameters: {'n_factors': 23, 'n_epochs': 50, 'lr_all': 0.09738861861565962, 'reg_all': 0.4681357053025206}. Best is trial 0 with value: 1.6157304932293513.
[I 2024-08-29 17:52:11,869] Trial 2 finished with value: 1.6159778115801442 and parameters: {'n_factors': 11, 'n_epochs': 28, 'lr_all': 0.019748666872682532, 'reg_all': 0.9883608511252329}. Best is trial 0 with value: 1.6157304932293513.
[I 2024-08-29 17:52:15,142] Trial 3 finished with value: 1.625288923071538 and parameters: {'n_factors': 82, 'n_epochs': 18, 'lr_all': 0.07803444743282961, 'reg_all': 0.8338866

In [44]:
best_params = study.best_trials[0].params
best_params

{'n_factors': 43,
 'n_epochs': 36,
 'lr_all': 0.0012335145480055056,
 'reg_all': 0.7675430574687598}

In [49]:
model = SVD(
    n_factors = 43,
    n_epochs = 36,
    lr_all = 0.0012335145480055056,
    reg_all = 0.7675430574687598
)

In [50]:
model.fit(trainset)

In [51]:
preds = model.test(testset)

print(f"mae = {accuracy.mae(preds, verbose=False)}")
print(f"mse = {accuracy.mse(preds, verbose=False)}")
print(f"rmse= {accuracy.rmse(preds, verbose=False)}")
print(f"fcp = {accuracy.fcp(preds, verbose=False)}")

mae = 1.6139470170137307
mse = 3.171409456887831
rmse= 1.7808451524171973
fcp = 0.5043455240514976
